In [1]:
#///// imports

import os
import cvxpy as cp
import numpy as np

import time

from grand_solver import grand_solver as gs


from PFS_update_profile_General import PFS_update_profile as pfs

output_folder = 'checkpoints'


# This command creates the folder in the same directory as your script
os.makedirs(output_folder, exist_ok=True) 

In [2]:
# initializations

operation_length = 200

base_freq=1e9
bandwidth = 200e3
block_duration = 60



sigma2 = 1e-14 # noise power -110dBm
H = 100  # UAV Height
max_speed = 250 #10 second transition and 25m/s speed
min_dist = 100
power = 1e-7


B = 2  # number of frequency bands
K = 10  # number of users
#M = 3  # number of UAVs !!! set M<K
M = 2


Lipshitz1 = 1.62
Lipshitz2 = (K*B)**(0.5) + 0.01

max_loc = 550

plot_offset = max_loc

# Initialize backhaul
backhaul = np.array([[-1000,-1000,1000,1000], [-1000,1000,-1000,1000]])
D = backhaul.shape[1]  # Get number of columns
back_power = 2e-7

#(not) scaled:
back_power = 4e-7;
pow_lim = back_power;
pow_init = (1e-8)*(1e-7)*np.ones((M,B+1))
pow_init[:,-1] = 0.1*(1e-7)
sigma2 = 1e-14; #multiplied by 1






np.random.seed(345)
back_initial = (1/D)* np.ones((M, D))

#back_initial = np.random.rand(M, D)
#back_initial = back_initial / np.sum(back_initial)

initial_sched = (1/M/B) *  np.ones((B, M, K))

#initial_sched = np.random.rand(B, M, K)
#initial_sched = initial_sched / np.sum(initial_sched)
#initial_sched = 0.3 * np.ones((M, K, B))

# User profile parameters
profile_update_rate = 0.2
user_profile = np.ones(K)









# Set random seeds and generate user locations

np.random.seed(30)

# The following block generates random locations but is immediately
# overwritten by a hardcoded value, just like in the MATLAB script.
# This is included to be a perfect 1-to-1 conversion.
low = int(-max_loc / 3)
high = int(max_loc / 3)
# Note: numpy's randint high is exclusive, so we add 1
x_random = np.random.randint(low, high + 1, size=(2, K // 2)) - np.array([[max_loc + 100], [0]])

# Hardcoded location for the first set of users
x = np.array([
    [-950, -800, -600, 0 , 380],
    [150, 800, -700 , -950 , -970]
])

# Set a new seed for the next random generation
np.random.seed(345)

# This block is also overwritten by the hardcoded value below.
low = int(-max_loc / 1.5)
high = int(max_loc / 1.5)
y_random = np.random.randint(low, high + 1, size=(2, K // 2)) + max_loc

# Hardcoded location for the second set of users
y = np.array([
    [10, 900, 620, -250 , 250],
    [10, -200, 500 , 750 , 950]
])

# Concatenate the two sets of user locations horizontally (side-by-side)
user_loc = np.concatenate((x, y), axis=1)


# --- UAV Location Setup ---

# Set seed and generate random x-coordinates
np.random.seed(3)
x_uav = np.random.randint(-max_loc, max_loc + 1, size=(1, M))

# Set seed and generate random y-coordinates
np.random.seed(5)
y_uav = np.random.randint(-max_loc, max_loc + 1, size=(1, M))

# Concatenate the random coordinates vertically to form UAV locations
UAV_loc_random = np.concatenate((x_uav, y_uav), axis=0)

# The MATLAB script comments out several options and settles on a final one.
# We will do the same. This is the final value that will be used.
# UAV_loc = np.array([[250], [0]])
# UAV_loc = np.array([[150, 150], [-150, -150]])
# UAV_loc = np.array([[150], [-150]])
UAV_loc = np.array([
    [-50, 50, 0],
    [50, 50, -50]
])

UAV_loc = np.array([
    [-50,50],
    [0,0]
])


# UAV_loc = np.array([
#     [0],
#     [0]
# ])


















#scaling : 


# # Define your scaling factors once at the beginning
power_scale = 1e-11  # We will divide all power-related values by this
dist_scale = 100.0    # We will divide all distance-related values by this


# --- Scaled Inputs ---
H = H / dist_scale  # This will be 1.0
max_speed = max_speed / dist_scale
min_dist = min_dist / dist_scale

user_loc = user_loc / dist_scale
UAV_loc = UAV_loc / dist_scale
backhaul = backhaul / dist_scale

pow_lim = pow_lim / power_scale/dist_scale**2
back_power = back_power / power_scale/dist_scale**2 # This will be 1.0
pow_init = pow_init / power_scale/dist_scale**2

sigma2 = sigma2 / power_scale

In [ ]:
# check values

print(H)
print(max_speed)
print(min_dist)
print(user_loc)
print(UAV_loc)
print(backhaul)
print(pow_lim)
print(back_power)
print(pow_init)
print(sigma2)


In [ ]:
# use if you want load from a specific timestep
counter = 6

data = np.load(os.path.join(output_folder, f'checkpoint{counter}.npz'))
print(data['user_profile'])

print(data['UAV_loc'])




UAV_loc = data['UAV_loc']
user_profile = data['user_profile']
data.close()

In [ ]:


counter = 0       # to store frames if needed

str_block = "========================================================";

rate_tot = 0

while counter < operation_length:
    start_time = time.time()  # tic

    # Call optimization function
    sched, back_sched, power, pos, val, rate = gs(Lipshitz1,Lipshitz2,user_loc.copy(),UAV_loc.copy(),initial_sched.copy(),user_profile.copy(),sigma2,H,B,backhaul.copy(),back_power,back_initial.copy(),max_speed,min_dist,pow_init.copy(),pow_lim)

    # Create the full path to the file


    # Save your data using the path


    back = back_sched
    X = (sched)

    UAV_loc = pos

    sum_rate = np.sum(rate)

    user_profile , _ = pfs(X, UAV_loc, user_loc, sigma2, H, B, user_profile.copy(), profile_update_rate,bandwidth,power)
    #user_profile , rate = pfs(sched, UAV_loc, user_loc, sigma2, H, B, user_profile, profile_update_rate,bandwidth,power)

    file_path = os.path.join(output_folder, f'checkpoint{counter+1}.npz')
    np.savez_compressed(file_path, sched=sched, UAV_loc=UAV_loc,power = power, user_profile=user_profile, rate=rate, back=back_sched, val=val, sum_rate = sum_rate)

    
    rate_tot +=np.sum(rate)

    # data1 = np.load(os.path.join(output_folder1, f'checkpoint{counter+1}.npz'))
    # UAV_loc = data1['UAV_loc']
    # user_profile = data1['user_profile']
    # data1.close()


    # Print debug info (like MATLAB command window output)
    print(str_block)
    print("Iteration", counter+1)
    print("function value =", val)
    print("sum_rate =", sum_rate)
    print("total_rate = ", rate_tot)
    # print("scheduling =", np.round(X,decimals=3),"\n",np.round(X*power.T[...,None],decimals=3))
    print("scheduling =", np.round(X,decimals=3),"\n")
    print("UAV_loc =", np.round(UAV_loc,decimals=3))
    print("back_scheduling =", np.round(back,decimals=3))
    print("allocated power =", np.round(power,decimals=3),np.sum(power))
    print("Rate =", rate)
    print("profile =", user_profile)

    counter += 1

    # toc
    end_time = time.time()
    print(f"Iteration time: {end_time - start_time:.3f}s")


C:\Users\Digikala\anaconda3\envs\my_thesis\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:41: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


val_func4:
 7.486670194683928 10007.486670194683

--- Iteration: 1 ---
val_func4:
 11.496219116163573 4.009548921479645

--- Iteration: 2 ---
val_func4:
 12.30947939871563 0.8132602825520578

--- Iteration: 3 ---
val_func4:
 27.371329103356413 15.061849704640782

--- Iteration: 4 ---
val_func4:
 42.66290854278936 15.291579439432947

--- Iteration: 5 ---
val_func4:
 42.910102989179364 0.24719444639000443

--- Iteration: 6 ---
val_func4:
 43.125312308794285 0.21520931961492096

--- Iteration: 7 ---
val_func4:
 43.33245945452786 0.20714714573357185

--- Iteration: 8 ---
val_func4:
 43.510659493237576 0.17820003870971846

--- Iteration: 9 ---
val_func4:
 43.67261019422895 0.1619507009913761

--- Iteration: 10 ---
val_func4:
 43.821856138217484 0.1492459439885323

--- Iteration: 11 ---
val_func4:
 43.958628570537726 0.1367724323202424

--- Iteration: 12 ---
val_func4:
 44.08273697064061 0.12410840010288382

--- Iteration: 13 ---
val_func4:
 44.19474226724228 0.11200529660167291

--- Iterati

In [ ]:
x = np.array([
    [-950, -800, -600, 0 , 380],
    [150, 800, -700 , -950 , -970]
])
y = np.array([
    [10, 900, 620, -250 , 250],
    [10, -200, 500 , 750 , 950]
])


In [ ]:
# assessment
